In [0]:
import tensorflow as tf
import numpy as np
import keras

from keras import optimizers, regularizers
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Input
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator

# from tflearn.datasets import cifar10, cifar100

In [0]:
# Initialize parameters for the model
width = 32 # width of image in pixels
height = 32  # height of the image in pixels
n_inputs = width*height # number of pixels in one image/ number of inputs
n_classes = 10 # There are 10 possible classifications for this problem

batch_size = 128
epochs = 200
learning_rate = 0.005
reg_rate = 1e-4

In [0]:
class get_cifar_data(object):
    def __init__(self,n_classes):
        (xtrain,ytrain),(self.X_test,self.Y_test) = cifar10.load_data()
        self.n_classes = n_classes
        self.X_val = xtrain[:5000].astype('float32')
        self.Y_val = ytrain[:5000]
        self.X_train = xtrain[5000:].astype('float32')
        self.Y_train = ytrain[5000:]
        self.X_test = self.X_test.astype('float32')
        self.Y_test = self.Y_test
        
        self.X_test = self.normalize(self.X_test)
        self.X_train = self.normalize(self.X_train)
        self.X_val = self.normalize(self.X_val)
        self.Y_train = self.get_one_hot(self.Y_train,self.n_classes)
        self.Y_val = self.get_one_hot(self.Y_val,self.n_classes)
        self.Y_test = self.get_one_hot(self.Y_test,self.n_classes)

    # https://stackoverflow.com/questions/38592324/one-hot-encoding-using-numpy/38592416
    def get_one_hot(self, data,n_classes):
        targets = np.array(data).reshape(-1)
        return np.eye(n_classes)[targets]
    # From here:
    # (https://medium.com/@lahorekid/a-dummies-guide-to-data-normalization-for-neural-nets-ff1998116e75)
    # I confirmed that normalized data converges faster during backpropagation
    def normalize(self, X):
        mean = np.mean(X, axis=(0, 1, 2, 3))
        std = np.std(X, axis=(0, 1, 2, 3))
        X = (X-mean)/(std+1e-7)
        return X

In [0]:
class CNNModel(object):
    def __init__(self,data,batch_size,epochs, learning_rate, reg_rate):
        self.data = data
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.reg_rate = reg_rate
        self.cnn_model_fn()

Deep Residual Networks Inspiration: 

* https://blog.waya.ai/deep-residual-learning-9610bb62c355
* https://arxiv.org/pdf/1512.03385.pdf

This was implemented using the older/test tf.contrib module and gave best top1 accuracy of 0.74
So I no longer used it and based on the advice of some peers started using keras with simple conv block structures.



In [0]:
class CNNModel(CNNModel):   
    def residual_block(self, x, depth, kernel):
        residual = x
        residual = tf.contrib.layers.batch_norm(residual, activation_fn=tf.nn.relu)
        residual = tf.contrib.layers.conv2d(residual, depth, kernel)
        residual = tf.contrib.layers.batch_norm(residual, activation_fn=tf.nn.relu)
        residual = tf.contrib.layers.conv2d(residual, x.get_shape()[3], kernel)
        residual = residual + x
        return residual
  
    def residual_network(self,
                         x,
                         num_classes=None,
                         global_pool=True,
                         output_stride=None,
                         reuse=None):
        net = x
        net = tf.contrib.layers.repeat(net, 1, tf.contrib.layers.conv2d, 32, [5, 5], stride=1)
        net = tf.contrib.layers.max_pool2d(net, [2, 2])
        net = tf.contrib.layers.repeat(net, 1, tf.contrib.layers.conv2d, 32, [5, 5], stride=1)
        net = tf.contrib.layers.max_pool2d(net, [2, 2])
        net = self.residual_block(net, 32, [3, 3])
        net = self.residual_block(net, 32, [3, 3])
        net = self.residual_block(net, 64, [3, 3])
        net = self.residual_block(net, 64, [3, 3])
        net = tf.contrib.layers.flatten(net)
        net = tf.contrib.layers.fully_connected(net, 1024)
        net = tf.contrib.layers.dropout(net, 0.5)
        net = tf.contrib.layers.fully_connected(net, num_classes, activation_fn=None, normalizer_fn=None)
        return net

In [0]:
class CNNModel(CNNModel):
    def conv_block(self,filters,dropout,input):
        x = Conv2D(filters,[8,8],strides=1,padding='same',activation='relu', kernel_regularizer=regularizers.l2(self.reg_rate))(input) # 
        x = BatchNormalization()(x)
        x = Conv2D(filters,[4,4],strides=1,padding='same',activation='relu', kernel_regularizer=regularizers.l2(self.reg_rate))(input) #
        x = BatchNormalization()(x)
        x = MaxPooling2D(2,strides=2)(x)
        x = Dropout(dropout)(x)
        return x

    def cnn_model_fn(self):
        input = Input(shape = self.data.X_train.shape[1:])
        x = self.conv_block(32,0.3,input)        
        x = self.conv_block(64,0.4,x)
        x = self.conv_block(128,0.5,x)
        x = self.conv_block(256,0.3,x)
        x = Flatten()(x)
        x = Dense(1024,activation = 'relu')(x)
        y = Dense(self.data.n_classes, activation='softmax')(x)
        self.model = Model(inputs = input, outputs = y)

In [0]:
class CNNModel(CNNModel):    
    def LR_SCHEDULE(self, epoch):
        lr = self.learning_rate
        if epoch > 100:
            lr = 0.000625
        elif epoch > 80:
            lr = 0.00125
        elif epoch > 60:
            lr = 0.0025
        elif epoch > 40:
            lr = 0.005
        elif epoch > 20:
            lr = 0.0075
    
    # Trying a decaying learning rate
#         decay = 0.01
#         lr = self.learning_rate*1/(1 + decay*epoch)
#         print(lr)

        return lr

    def train(self):
        self.optimizer = optimizers.Adam(self.learning_rate)
# I tried other optimizers such as those below but had no luck
#         self.optimizer = optimizer.RMSProp(self.learning_rate)
#         self.optimizer = optimizer.Adadelta()
        self.model.compile(self.optimizer,'categorical_crossentropy',['accuracy'])

        self.datagen = keras.preprocessing.image.ImageDataGenerator(
            horizontal_flip = True,fill_mode = 'constant',
            width_shift_range = 4, height_shift_range = 4
            )
        self.datagen.fit(self.data.X_train)
        self.model.fit_generator(self.datagen.flow(self.data.X_train,
            self.data.Y_train,batch_size=self.batch_size),
            steps_per_epoch=len(self.data.X_train)/self.batch_size,
            epochs=self.epochs,
            validation_data=(self.data.X_val,self.data.Y_val),
            callbacks=[LearningRateScheduler(self.LR_SCHEDULE)],verbose=2)

In [24]:
data = get_cifar_data(n_classes)
m = CNNModel(data, batch_size, epochs, learning_rate, reg_rate)
m.train()
scores = m.model.evaluate(data.X_test,data.Y_test,verbose=2)
print('\nTest result: %.3f loss: %.3f' % (scores[1],scores[0]))

Epoch 1/200
 - 28s - loss: 2.7269 - acc: 0.2768 - val_loss: 1.7897 - val_acc: 0.3936
Epoch 2/200
 - 25s - loss: 1.7896 - acc: 0.3903 - val_loss: 1.6715 - val_acc: 0.4444
Epoch 3/200
 - 26s - loss: 1.6563 - acc: 0.4525 - val_loss: 1.4533 - val_acc: 0.5318
Epoch 4/200
 - 25s - loss: 1.5879 - acc: 0.4966 - val_loss: 1.4720 - val_acc: 0.5586
Epoch 5/200
 - 26s - loss: 1.5291 - acc: 0.5315 - val_loss: 1.2853 - val_acc: 0.6248
Epoch 6/200
 - 25s - loss: 1.4781 - acc: 0.5618 - val_loss: 1.2651 - val_acc: 0.6522
Epoch 7/200
 - 25s - loss: 1.4307 - acc: 0.5901 - val_loss: 1.2825 - val_acc: 0.6568
Epoch 8/200
 - 25s - loss: 1.4089 - acc: 0.6091 - val_loss: 1.2166 - val_acc: 0.6838
Epoch 9/200
 - 25s - loss: 1.3878 - acc: 0.6273 - val_loss: 1.2332 - val_acc: 0.6866
Epoch 10/200
 - 25s - loss: 1.3870 - acc: 0.6346 - val_loss: 1.2929 - val_acc: 0.6754
Epoch 11/200
 - 25s - loss: 1.3686 - acc: 0.6509 - val_loss: 1.2131 - val_acc: 0.7098
Epoch 12/200
 - 26s - loss: 1.3691 - acc: 0.6530 - val_loss: 1.

## Best Result

Epoch 1/200
 - 28s - loss: 2.7269 - acc: 0.2768 - val_loss: 1.7897 - val_acc: 0.3936
Epoch 2/200
 - 25s - loss: 1.7896 - acc: 0.3903 - val_loss: 1.6715 - val_acc: 0.4444
Epoch 3/200
 - 26s - loss: 1.6563 - acc: 0.4525 - val_loss: 1.4533 - val_acc: 0.5318
Epoch 4/200
 - 25s - loss: 1.5879 - acc: 0.4966 - val_loss: 1.4720 - val_acc: 0.5586
Epoch 5/200
 - 26s - loss: 1.5291 - acc: 0.5315 - val_loss: 1.2853 - val_acc: 0.6248
Epoch 6/200
 - 25s - loss: 1.4781 - acc: 0.5618 - val_loss: 1.2651 - val_acc: 0.6522
Epoch 7/200
 - 25s - loss: 1.4307 - acc: 0.5901 - val_loss: 1.2825 - val_acc: 0.6568
Epoch 8/200
 - 25s - loss: 1.4089 - acc: 0.6091 - val_loss: 1.2166 - val_acc: 0.6838
Epoch 9/200
 - 25s - loss: 1.3878 - acc: 0.6273 - val_loss: 1.2332 - val_acc: 0.6866
Epoch 10/200
 - 25s - loss: 1.3870 - acc: 0.6346 - val_loss: 1.2929 - val_acc: 0.6754
Epoch 11/200
 - 25s - loss: 1.3686 - acc: 0.6509 - val_loss: 1.2131 - val_acc: 0.7098
Epoch 12/200
 - 26s - loss: 1.3691 - acc: 0.6530 - val_loss: 1.2265 - val_acc: 0.7066
Epoch 13/200
 - 25s - loss: 1.3599 - acc: 0.6616 - val_loss: 1.1745 - val_acc: 0.7360
Epoch 14/200
 - 25s - loss: 1.3629 - acc: 0.6660 - val_loss: 1.1902 - val_acc: 0.7264
Epoch 15/200
 - 26s - loss: 1.3525 - acc: 0.6742 - val_loss: 1.2673 - val_acc: 0.7142
Epoch 16/200
 - 25s - loss: 1.3476 - acc: 0.6756 - val_loss: 1.2368 - val_acc: 0.7164
Epoch 17/200
 - 26s - loss: 1.3401 - acc: 0.6828 - val_loss: 1.1636 - val_acc: 0.7410
Epoch 18/200
 - 25s - loss: 1.3335 - acc: 0.6836 - val_loss: 1.1777 - val_acc: 0.7456
Epoch 19/200
 - 26s - loss: 1.3423 - acc: 0.6822 - val_loss: 1.2232 - val_acc: 0.7266
Epoch 20/200
 - 25s - loss: 1.3490 - acc: 0.6848 - val_loss: 1.1457 - val_acc: 0.7538
Epoch 21/200
 - 26s - loss: 1.3311 - acc: 0.6903 - val_loss: 1.1487 - val_acc: 0.7638
Epoch 22/200
 - 26s - loss: 1.4824 - acc: 0.6704 - val_loss: 1.3828 - val_acc: 0.7178
Epoch 23/200
 - 26s - loss: 1.5138 - acc: 0.6658 - val_loss: 1.3970 - val_acc: 0.7100
Epoch 24/200
 - 26s - loss: 1.5124 - acc: 0.6680 - val_loss: 1.3849 - val_acc: 0.7118
Epoch 25/200
 - 25s - loss: 1.5088 - acc: 0.6644 - val_loss: 1.3540 - val_acc: 0.7156
Epoch 26/200
 - 25s - loss: 1.5108 - acc: 0.6704 - val_loss: 1.2949 - val_acc: 0.7482
Epoch 27/200
 - 26s - loss: 1.5120 - acc: 0.6697 - val_loss: 1.3623 - val_acc: 0.7266
Epoch 28/200
 - 25s - loss: 1.5102 - acc: 0.6692 - val_loss: 1.3126 - val_acc: 0.7358
Epoch 29/200
 - 25s - loss: 1.5085 - acc: 0.6735 - val_loss: 1.3960 - val_acc: 0.7100
Epoch 30/200
 - 25s - loss: 1.5069 - acc: 0.6715 - val_loss: 1.3982 - val_acc: 0.7128
Epoch 31/200
 - 25s - loss: 1.4956 - acc: 0.6725 - val_loss: 1.3648 - val_acc: 0.7210
Epoch 32/200
 - 25s - loss: 1.5087 - acc: 0.6712 - val_loss: 1.3263 - val_acc: 0.7376
Epoch 33/200
 - 25s - loss: 1.4908 - acc: 0.6780 - val_loss: 1.3765 - val_acc: 0.7186
Epoch 34/200
 - 25s - loss: 1.5006 - acc: 0.6715 - val_loss: 1.3505 - val_acc: 0.7272
Epoch 35/200
 - 26s - loss: 1.4915 - acc: 0.6743 - val_loss: 1.3985 - val_acc: 0.7142
Epoch 36/200
 - 26s - loss: 1.4776 - acc: 0.6755 - val_loss: 1.3919 - val_acc: 0.7038
Epoch 37/200
 - 26s - loss: 1.4860 - acc: 0.6763 - val_loss: 1.2635 - val_acc: 0.7514
Epoch 38/200
 - 25s - loss: 1.4829 - acc: 0.6741 - val_loss: 1.3256 - val_acc: 0.7236
Epoch 39/200
 - 26s - loss: 1.4756 - acc: 0.6766 - val_loss: 1.3335 - val_acc: 0.7312
Epoch 40/200
 - 25s - loss: 1.4740 - acc: 0.6770 - val_loss: 1.4139 - val_acc: 0.6976
Epoch 41/200
 - 25s - loss: 1.4687 - acc: 0.6754 - val_loss: 1.3062 - val_acc: 0.7348
Epoch 42/200
 - 25s - loss: 1.3492 - acc: 0.6984 - val_loss: 1.2068 - val_acc: 0.7280
Epoch 43/200
 - 25s - loss: 1.2962 - acc: 0.7031 - val_loss: 1.1159 - val_acc: 0.7686
Epoch 44/200
 - 25s - loss: 1.2888 - acc: 0.7016 - val_loss: 1.1144 - val_acc: 0.7644
Epoch 45/200
 - 25s - loss: 1.2819 - acc: 0.7023 - val_loss: 1.2049 - val_acc: 0.7324
Epoch 46/200
 - 25s - loss: 1.2774 - acc: 0.7044 - val_loss: 1.0963 - val_acc: 0.7730
Epoch 47/200
 - 25s - loss: 1.2660 - acc: 0.7075 - val_loss: 1.0585 - val_acc: 0.7832
Epoch 48/200
 - 25s - loss: 1.2653 - acc: 0.7073 - val_loss: 1.0604 - val_acc: 0.7804
Epoch 49/200
 - 26s - loss: 1.2644 - acc: 0.7073 - val_loss: 1.0937 - val_acc: 0.7714
Epoch 50/200
 - 25s - loss: 1.2636 - acc: 0.7092 - val_loss: 1.1231 - val_acc: 0.7574
Epoch 51/200
 - 25s - loss: 1.2653 - acc: 0.7060 - val_loss: 1.1091 - val_acc: 0.7640
Epoch 52/200
 - 25s - loss: 1.2579 - acc: 0.7093 - val_loss: 1.0699 - val_acc: 0.7764
Epoch 53/200
 - 26s - loss: 1.2579 - acc: 0.7092 - val_loss: 1.0711 - val_acc: 0.7710
Epoch 54/200
 - 26s - loss: 1.2503 - acc: 0.7094 - val_loss: 1.1593 - val_acc: 0.7470
Epoch 55/200
 - 26s - loss: 1.2558 - acc: 0.7114 - val_loss: 1.1025 - val_acc: 0.7560
Epoch 56/200
 - 25s - loss: 1.2382 - acc: 0.7133 - val_loss: 1.1127 - val_acc: 0.7610
Epoch 57/200
 - 25s - loss: 1.2421 - acc: 0.7123 - val_loss: 1.1336 - val_acc: 0.7556
Epoch 58/200
 - 25s - loss: 1.2424 - acc: 0.7146 - val_loss: 1.1543 - val_acc: 0.7444
Epoch 59/200
 - 25s - loss: 1.2435 - acc: 0.7173 - val_loss: 1.1504 - val_acc: 0.7518
Epoch 60/200
 - 25s - loss: 1.2462 - acc: 0.7103 - val_loss: 1.1310 - val_acc: 0.7522
Epoch 61/200
 - 26s - loss: 1.2435 - acc: 0.7123 - val_loss: 1.0794 - val_acc: 0.7676
Epoch 62/200
 - 25s - loss: 1.1294 - acc: 0.7398 - val_loss: 0.9862 - val_acc: 0.7818
Epoch 63/200
 - 26s - loss: 1.0676 - acc: 0.7426 - val_loss: 0.9046 - val_acc: 0.7980
Epoch 64/200
 - 25s - loss: 1.0445 - acc: 0.7480 - val_loss: 0.8922 - val_acc: 0.7974
Epoch 65/200
 - 26s - loss: 1.0340 - acc: 0.7464 - val_loss: 0.8713 - val_acc: 0.8046
Epoch 66/200
 - 25s - loss: 1.0252 - acc: 0.7479 - val_loss: 0.8288 - val_acc: 0.8146
Epoch 67/200
 - 26s - loss: 1.0129 - acc: 0.7495 - val_loss: 0.8870 - val_acc: 0.7942
Epoch 68/200
 - 26s - loss: 1.0013 - acc: 0.7539 - val_loss: 0.8471 - val_acc: 0.8082
Epoch 69/200
 - 26s - loss: 1.0008 - acc: 0.7532 - val_loss: 0.8556 - val_acc: 0.8022
Epoch 70/200
 - 26s - loss: 1.0025 - acc: 0.7510 - val_loss: 0.8671 - val_acc: 0.8036
Epoch 71/200
 - 26s - loss: 0.9991 - acc: 0.7531 - val_loss: 0.8305 - val_acc: 0.8126
Epoch 72/200
 - 25s - loss: 0.9958 - acc: 0.7534 - val_loss: 0.8524 - val_acc: 0.8018
Epoch 73/200
 - 27s - loss: 0.9902 - acc: 0.7562 - val_loss: 0.8318 - val_acc: 0.8122
Epoch 74/200
 - 26s - loss: 0.9869 - acc: 0.7564 - val_loss: 0.8188 - val_acc: 0.8160
Epoch 75/200
 - 26s - loss: 0.9831 - acc: 0.7562 - val_loss: 0.8322 - val_acc: 0.8114
Epoch 76/200
 - 26s - loss: 0.9874 - acc: 0.7567 - val_loss: 0.8035 - val_acc: 0.8204
Epoch 77/200
 - 27s - loss: 0.9842 - acc: 0.7550 - val_loss: 0.8153 - val_acc: 0.8124
Epoch 78/200
 - 26s - loss: 0.9836 - acc: 0.7568 - val_loss: 0.8328 - val_acc: 0.8074
Epoch 79/200
 - 26s - loss: 0.9858 - acc: 0.7541 - val_loss: 0.7929 - val_acc: 0.8226
Epoch 80/200
 - 25s - loss: 0.9816 - acc: 0.7582 - val_loss: 0.8212 - val_acc: 0.8124
Epoch 81/200
 - 26s - loss: 0.9776 - acc: 0.7570 - val_loss: 0.8073 - val_acc: 0.8182
Epoch 82/200
 - 26s - loss: 0.9331 - acc: 0.7695 - val_loss: 0.7557 - val_acc: 0.8334
Epoch 83/200
 - 26s - loss: 0.8924 - acc: 0.7768 - val_loss: 0.7476 - val_acc: 0.8294
Epoch 84/200
 - 26s - loss: 0.8795 - acc: 0.7779 - val_loss: 0.7588 - val_acc: 0.8246
Epoch 85/200
 - 27s - loss: 0.8680 - acc: 0.7797 - val_loss: 0.7042 - val_acc: 0.8360
Epoch 86/200
 - 26s - loss: 0.8501 - acc: 0.7824 - val_loss: 0.6836 - val_acc: 0.8410
Epoch 87/200
 - 26s - loss: 0.8443 - acc: 0.7828 - val_loss: 0.6915 - val_acc: 0.8348
Epoch 88/200
 - 26s - loss: 0.8377 - acc: 0.7828 - val_loss: 0.6814 - val_acc: 0.8390
Epoch 89/200
 - 26s - loss: 0.8304 - acc: 0.7842 - val_loss: 0.6860 - val_acc: 0.8386
Epoch 90/200
 - 26s - loss: 0.8301 - acc: 0.7839 - val_loss: 0.6860 - val_acc: 0.8340
Epoch 91/200
 - 26s - loss: 0.8267 - acc: 0.7825 - val_loss: 0.6695 - val_acc: 0.8368
Epoch 92/200
 - 26s - loss: 0.8201 - acc: 0.7850 - val_loss: 0.6723 - val_acc: 0.8366
Epoch 93/200
 - 27s - loss: 0.8198 - acc: 0.7861 - val_loss: 0.6566 - val_acc: 0.8422
Epoch 94/200
 - 26s - loss: 0.8123 - acc: 0.7866 - val_loss: 0.6666 - val_acc: 0.8358
Epoch 95/200
 - 27s - loss: 0.8133 - acc: 0.7856 - val_loss: 0.6756 - val_acc: 0.8346
Epoch 96/200
 - 26s - loss: 0.8107 - acc: 0.7875 - val_loss: 0.6728 - val_acc: 0.8346
Epoch 97/200
 - 27s - loss: 0.8136 - acc: 0.7847 - val_loss: 0.6516 - val_acc: 0.8448
Epoch 98/200
 - 26s - loss: 0.8054 - acc: 0.7872 - val_loss: 0.6579 - val_acc: 0.8394
Epoch 99/200
 - 27s - loss: 0.8080 - acc: 0.7857 - val_loss: 0.6558 - val_acc: 0.8458
Epoch 100/200
 - 26s - loss: 0.7989 - acc: 0.7911 - val_loss: 0.6457 - val_acc: 0.8452
Epoch 101/200
 - 26s - loss: 0.8021 - acc: 0.7899 - val_loss: 0.6477 - val_acc: 0.8456
Epoch 102/200
 - 26s - loss: 0.7737 - acc: 0.7982 - val_loss: 0.6224 - val_acc: 0.8486
Epoch 103/200
 - 26s - loss: 0.7571 - acc: 0.8009 - val_loss: 0.6149 - val_acc: 0.8522
Epoch 104/200
 - 26s - loss: 0.7531 - acc: 0.8006 - val_loss: 0.6353 - val_acc: 0.8406
Epoch 105/200
 - 26s - loss: 0.7472 - acc: 0.8015 - val_loss: 0.5954 - val_acc: 0.8588
Epoch 106/200
 - 26s - loss: 0.7316 - acc: 0.8056 - val_loss: 0.5961 - val_acc: 0.8524
Epoch 107/200
 - 26s - loss: 0.7278 - acc: 0.8047 - val_loss: 0.5879 - val_acc: 0.8542
Epoch 108/200
 - 26s - loss: 0.7315 - acc: 0.8028 - val_loss: 0.5917 - val_acc: 0.8502
Epoch 109/200
 - 26s - loss: 0.7212 - acc: 0.8048 - val_loss: 0.5722 - val_acc: 0.8584
Epoch 110/200
 - 25s - loss: 0.7187 - acc: 0.8056 - val_loss: 0.5836 - val_acc: 0.8508
Epoch 111/200
 - 26s - loss: 0.7089 - acc: 0.8080 - val_loss: 0.5839 - val_acc: 0.8540
Epoch 112/200
 - 25s - loss: 0.7079 - acc: 0.8078 - val_loss: 0.5706 - val_acc: 0.8572
Epoch 113/200
 - 25s - loss: 0.7086 - acc: 0.8091 - val_loss: 0.5729 - val_acc: 0.8574
Epoch 114/200
 - 25s - loss: 0.6987 - acc: 0.8108 - val_loss: 0.5772 - val_acc: 0.8518
Epoch 115/200
 - 25s - loss: 0.7011 - acc: 0.8090 - val_loss: 0.5699 - val_acc: 0.8536
Epoch 116/200
 - 25s - loss: 0.7012 - acc: 0.8100 - val_loss: 0.5725 - val_acc: 0.8504
Epoch 117/200
 - 25s - loss: 0.6935 - acc: 0.8108 - val_loss: 0.5616 - val_acc: 0.8574
Epoch 118/200
 - 25s - loss: 0.6951 - acc: 0.8096 - val_loss: 0.5622 - val_acc: 0.8590
Epoch 119/200
 - 26s - loss: 0.6922 - acc: 0.8113 - val_loss: 0.5821 - val_acc: 0.8488
Epoch 120/200
 - 25s - loss: 0.6888 - acc: 0.8120 - val_loss: 0.5648 - val_acc: 0.8538
Epoch 121/200
 - 26s - loss: 0.6875 - acc: 0.8098 - val_loss: 0.5569 - val_acc: 0.8582
Epoch 122/200
 - 25s - loss: 0.6902 - acc: 0.8101 - val_loss: 0.5448 - val_acc: 0.8610
Epoch 123/200
 - 25s - loss: 0.6884 - acc: 0.8115 - val_loss: 0.5720 - val_acc: 0.8544
Epoch 124/200
 - 25s - loss: 0.6852 - acc: 0.8117 - val_loss: 0.5618 - val_acc: 0.8552
Epoch 125/200
 - 25s - loss: 0.6828 - acc: 0.8111 - val_loss: 0.5708 - val_acc: 0.8532
Epoch 126/200
 - 25s - loss: 0.6843 - acc: 0.8106 - val_loss: 0.5596 - val_acc: 0.8528
Epoch 127/200
 - 25s - loss: 0.6869 - acc: 0.8099 - val_loss: 0.5422 - val_acc: 0.8628
Epoch 128/200
 - 25s - loss: 0.6787 - acc: 0.8139 - val_loss: 0.5608 - val_acc: 0.8562
Epoch 129/200
 - 25s - loss: 0.6824 - acc: 0.8110 - val_loss: 0.5453 - val_acc: 0.8562
Epoch 130/200
 - 25s - loss: 0.6710 - acc: 0.8141 - val_loss: 0.5457 - val_acc: 0.8598
Epoch 131/200
 - 25s - loss: 0.6761 - acc: 0.8123 - val_loss: 0.5553 - val_acc: 0.8592
Epoch 132/200
 - 26s - loss: 0.6728 - acc: 0.8123 - val_loss: 0.5435 - val_acc: 0.8596
Epoch 133/200
 - 27s - loss: 0.6726 - acc: 0.8130 - val_loss: 0.5425 - val_acc: 0.8572
Epoch 134/200
 - 26s - loss: 0.6778 - acc: 0.8121 - val_loss: 0.5349 - val_acc: 0.8628
Epoch 135/200
 - 26s - loss: 0.6711 - acc: 0.8121 - val_loss: 0.5513 - val_acc: 0.8576
Epoch 136/200
 - 25s - loss: 0.6706 - acc: 0.8149 - val_loss: 0.5486 - val_acc: 0.8602
Epoch 137/200
 - 26s - loss: 0.6725 - acc: 0.8127 - val_loss: 0.5325 - val_acc: 0.8612
Epoch 138/200
 - 25s - loss: 0.6714 - acc: 0.8122 - val_loss: 0.5361 - val_acc: 0.8612
Epoch 139/200
 - 26s - loss: 0.6685 - acc: 0.8155 - val_loss: 0.5405 - val_acc: 0.8602
Epoch 140/200
 - 25s - loss: 0.6685 - acc: 0.8128 - val_loss: 0.5436 - val_acc: 0.8574
Epoch 141/200
 - 26s - loss: 0.6675 - acc: 0.8143 - val_loss: 0.5371 - val_acc: 0.8632
Epoch 142/200
 - 25s - loss: 0.6655 - acc: 0.8146 - val_loss: 0.5471 - val_acc: 0.8594
Epoch 143/200
 - 25s - loss: 0.6698 - acc: 0.8135 - val_loss: 0.5559 - val_acc: 0.8524
Epoch 144/200
 - 25s - loss: 0.6652 - acc: 0.8162 - val_loss: 0.5422 - val_acc: 0.8614
Epoch 145/200
 - 26s - loss: 0.6671 - acc: 0.8117 - val_loss: 0.5405 - val_acc: 0.8538
Epoch 146/200
 - 25s - loss: 0.6616 - acc: 0.8163 - val_loss: 0.5469 - val_acc: 0.8584
Epoch 147/200
 - 26s - loss: 0.6680 - acc: 0.8127 - val_loss: 0.5348 - val_acc: 0.8564
Epoch 148/200
 - 25s - loss: 0.6617 - acc: 0.8168 - val_loss: 0.5450 - val_acc: 0.8612
Epoch 149/200
 - 26s - loss: 0.6692 - acc: 0.8130 - val_loss: 0.5455 - val_acc: 0.8572
Epoch 150/200
 - 25s - loss: 0.6577 - acc: 0.8185 - val_loss: 0.5366 - val_acc: 0.8584
Epoch 151/200
 - 26s - loss: 0.6608 - acc: 0.8172 - val_loss: 0.5510 - val_acc: 0.8542
Epoch 152/200
 - 25s - loss: 0.6602 - acc: 0.8159 - val_loss: 0.5563 - val_acc: 0.8528
Epoch 153/200
 - 26s - loss: 0.6633 - acc: 0.8155 - val_loss: 0.5337 - val_acc: 0.8632
Epoch 154/200
 - 25s - loss: 0.6582 - acc: 0.8179 - val_loss: 0.5285 - val_acc: 0.8636
Epoch 155/200
 - 26s - loss: 0.6573 - acc: 0.8172 - val_loss: 0.5237 - val_acc: 0.8626
Epoch 156/200
 - 25s - loss: 0.6607 - acc: 0.8170 - val_loss: 0.5472 - val_acc: 0.8576
Epoch 157/200
 - 26s - loss: 0.6547 - acc: 0.8174 - val_loss: 0.5401 - val_acc: 0.8552
Epoch 158/200
 - 26s - loss: 0.6550 - acc: 0.8184 - val_loss: 0.5603 - val_acc: 0.8512
Epoch 159/200
 - 26s - loss: 0.6607 - acc: 0.8125 - val_loss: 0.5379 - val_acc: 0.8590
Epoch 160/200
 - 25s - loss: 0.6550 - acc: 0.8180 - val_loss: 0.5280 - val_acc: 0.8596
Epoch 161/200
 - 26s - loss: 0.6536 - acc: 0.8177 - val_loss: 0.5629 - val_acc: 0.8588
Epoch 162/200
 - 25s - loss: 0.6517 - acc: 0.8190 - val_loss: 0.5555 - val_acc: 0.8514
Epoch 163/200
 - 26s - loss: 0.6532 - acc: 0.8157 - val_loss: 0.5251 - val_acc: 0.8694
Epoch 164/200
 - 25s - loss: 0.6589 - acc: 0.8174 - val_loss: 0.5276 - val_acc: 0.8670
Epoch 165/200
 - 26s - loss: 0.6504 - acc: 0.8196 - val_loss: 0.5214 - val_acc: 0.8692
Epoch 166/200
 - 25s - loss: 0.6527 - acc: 0.8177 - val_loss: 0.5328 - val_acc: 0.8594
Epoch 167/200
 - 26s - loss: 0.6474 - acc: 0.8222 - val_loss: 0.5428 - val_acc: 0.8604
Epoch 168/200
 - 25s - loss: 0.6532 - acc: 0.8182 - val_loss: 0.5477 - val_acc: 0.8616
Epoch 169/200
 - 26s - loss: 0.6468 - acc: 0.8203 - val_loss: 0.5289 - val_acc: 0.8610
Epoch 170/200
 - 25s - loss: 0.6551 - acc: 0.8185 - val_loss: 0.5304 - val_acc: 0.8618
Epoch 171/200
 - 25s - loss: 0.6532 - acc: 0.8197 - val_loss: 0.5240 - val_acc: 0.8658
Epoch 172/200
 - 25s - loss: 0.6517 - acc: 0.8190 - val_loss: 0.5315 - val_acc: 0.8622
Epoch 173/200
 - 26s - loss: 0.6523 - acc: 0.8181 - val_loss: 0.5262 - val_acc: 0.8644
Epoch 174/200
 - 25s - loss: 0.6529 - acc: 0.8176 - val_loss: 0.5390 - val_acc: 0.8598
Epoch 175/200
 - 26s - loss: 0.6501 - acc: 0.8196 - val_loss: 0.5268 - val_acc: 0.8628
Epoch 176/200
 - 25s - loss: 0.6455 - acc: 0.8208 - val_loss: 0.5209 - val_acc: 0.8658
Epoch 177/200
 - 26s - loss: 0.6497 - acc: 0.8182 - val_loss: 0.5271 - val_acc: 0.8624
Epoch 178/200
 - 25s - loss: 0.6491 - acc: 0.8193 - val_loss: 0.5245 - val_acc: 0.8638
Epoch 179/200
 - 26s - loss: 0.6474 - acc: 0.8197 - val_loss: 0.5413 - val_acc: 0.8590
Epoch 180/200
 - 25s - loss: 0.6498 - acc: 0.8182 - val_loss: 0.5157 - val_acc: 0.8642
Epoch 181/200
 - 26s - loss: 0.6476 - acc: 0.8196 - val_loss: 0.5280 - val_acc: 0.8600
Epoch 182/200
 - 26s - loss: 0.6547 - acc: 0.8170 - val_loss: 0.5170 - val_acc: 0.8666
Epoch 183/200
 - 25s - loss: 0.6451 - acc: 0.8208 - val_loss: 0.5221 - val_acc: 0.8682
Epoch 184/200
 - 25s - loss: 0.6522 - acc: 0.8184 - val_loss: 0.5213 - val_acc: 0.8684
Epoch 185/200
 - 26s - loss: 0.6435 - acc: 0.8212 - val_loss: 0.5371 - val_acc: 0.8632
Epoch 186/200
 - 25s - loss: 0.6499 - acc: 0.8191 - val_loss: 0.5359 - val_acc: 0.8608
Epoch 187/200
 - 25s - loss: 0.6415 - acc: 0.8229 - val_loss: 0.5208 - val_acc: 0.8666
Epoch 188/200
 - 25s - loss: 0.6436 - acc: 0.8206 - val_loss: 0.5402 - val_acc: 0.8602
Epoch 189/200
 - 25s - loss: 0.6412 - acc: 0.8230 - val_loss: 0.5188 - val_acc: 0.8664
Epoch 190/200
 - 25s - loss: 0.6449 - acc: 0.8208 - val_loss: 0.5241 - val_acc: 0.8678
Epoch 191/200
 - 25s - loss: 0.6433 - acc: 0.8214 - val_loss: 0.5204 - val_acc: 0.8656
Epoch 192/200
 - 25s - loss: 0.6398 - acc: 0.8229 - val_loss: 0.5113 - val_acc: 0.8688
Epoch 193/200
 - 26s - loss: 0.6401 - acc: 0.8232 - val_loss: 0.5218 - val_acc: 0.8664
Epoch 194/200
 - 26s - loss: 0.6408 - acc: 0.8235 - val_loss: 0.5217 - val_acc: 0.8670
Epoch 195/200
 - 26s - loss: 0.6445 - acc: 0.8212 - val_loss: 0.5249 - val_acc: 0.8632
Epoch 196/200
 - 25s - loss: 0.6452 - acc: 0.8219 - val_loss: 0.5236 - val_acc: 0.8664
Epoch 197/200
 - 26s - loss: 0.6389 - acc: 0.8230 - val_loss: 0.5191 - val_acc: 0.8646
Epoch 198/200
 - 25s - loss: 0.6368 - acc: 0.8242 - val_loss: 0.5403 - val_acc: 0.8612
Epoch 199/200
 - 26s - loss: 0.6423 - acc: 0.8217 - val_loss: 0.5146 - val_acc: 0.8662
Epoch 200/200
 - 25s - loss: 0.6419 - acc: 0.8229 - val_loss: 0.5259 - val_acc: 0.8650

Test result: 0.856 loss: 0.561

## Past Data 

Downloading data from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
170500096/170498071 [==============================] - 52s 0us/step
Epoch 1/200
 - 30s - loss: 2.5113 - acc: 0.2845 - val_loss: 2.2960 - val_acc: 0.2708
Epoch 2/200
 - 26s - loss: 1.9804 - acc: 0.3903 - val_loss: 1.8676 - val_acc: 0.3920
Epoch 3/200
 - 27s - loss: 1.8888 - acc: 0.4392 - val_loss: 1.7530 - val_acc: 0.4960
Epoch 4/200
 - 26s - loss: 1.8930 - acc: 0.4645 - val_loss: 1.6293 - val_acc: 0.5378
Epoch 5/200
 - 27s - loss: 1.8046 - acc: 0.4942 - val_loss: 1.9748 - val_acc: 0.4286
Epoch 6/200
 - 26s - loss: 1.6998 - acc: 0.5175 - val_loss: 1.7952 - val_acc: 0.4728
Epoch 7/200
 - 27s - loss: 1.5590 - acc: 0.5443 - val_loss: 1.3774 - val_acc: 0.6152
Epoch 8/200
 - 26s - loss: 1.5302 - acc: 0.5567 - val_loss: 1.3303 - val_acc: 0.6340
Epoch 9/200
 - 27s - loss: 1.5136 - acc: 0.5730 - val_loss: 1.4990 - val_acc: 0.5958
Epoch 10/200
 - 26s - loss: 1.4870 - acc: 0.5885 - val_loss: 1.2676 - val_acc: 0.6678
Epoch 11/200
 - 27s - loss: 1.4684 - acc: 0.5988 - val_loss: 1.4911 - val_acc: 0.5990
Epoch 12/200
 - 26s - loss: 1.4547 - acc: 0.6066 - val_loss: 1.3386 - val_acc: 0.6412
Epoch 13/200
 - 27s - loss: 1.4345 - acc: 0.6147 - val_loss: 1.4353 - val_acc: 0.6228
Epoch 14/200
 - 26s - loss: 1.4245 - acc: 0.6236 - val_loss: 1.2884 - val_acc: 0.6784
Epoch 15/200
 - 27s - loss: 1.4074 - acc: 0.6336 - val_loss: 1.3706 - val_acc: 0.6620
Epoch 16/200
 - 26s - loss: 1.3974 - acc: 0.6387 - val_loss: 1.2335 - val_acc: 0.6948
Epoch 17/200
 - 27s - loss: 1.3892 - acc: 0.6472 - val_loss: 1.4131 - val_acc: 0.6462
Epoch 18/200
 - 26s - loss: 1.3868 - acc: 0.6502 - val_loss: 1.3054 - val_acc: 0.6830
Epoch 19/200
 - 28s - loss: 1.3826 - acc: 0.6544 - val_loss: 1.4625 - val_acc: 0.6376
Epoch 20/200
 - 26s - loss: 1.3756 - acc: 0.6597 - val_loss: 1.2230 - val_acc: 0.7188
Epoch 21/200
 - 27s - loss: 1.3642 - acc: 0.6606 - val_loss: 1.2357 - val_acc: 0.7090
Epoch 22/200
 - 26s - loss: 1.2389 - acc: 0.6962 - val_loss: 1.0203 - val_acc: 0.7710
Epoch 23/200
 - 27s - loss: 1.1489 - acc: 0.7141 - val_loss: 0.9991 - val_acc: 0.7570
Epoch 24/200
 - 26s - loss: 1.1097 - acc: 0.7171 - val_loss: 0.9244 - val_acc: 0.7774
Epoch 25/200
 - 27s - loss: 1.0668 - acc: 0.7241 - val_loss: 0.9043 - val_acc: 0.7752
Epoch 26/200
 - 26s - loss: 1.0355 - acc: 0.7282 - val_loss: 0.8505 - val_acc: 0.7910
Epoch 27/200
 - 28s - loss: 1.0053 - acc: 0.7328 - val_loss: 0.8473 - val_acc: 0.7866
Epoch 28/200
 - 26s - loss: 0.9861 - acc: 0.7348 - val_loss: 0.8306 - val_acc: 0.7904
Epoch 29/200
 - 27s - loss: 0.9651 - acc: 0.7395 - val_loss: 0.8403 - val_acc: 0.7834
Epoch 30/200
 - 26s - loss: 0.9458 - acc: 0.7440 - val_loss: 0.7908 - val_acc: 0.7938
Epoch 31/200
 - 27s - loss: 0.9317 - acc: 0.7453 - val_loss: 0.7765 - val_acc: 0.7980
Epoch 32/200
 - 26s - loss: 0.9207 - acc: 0.7449 - val_loss: 0.7887 - val_acc: 0.7918
Epoch 33/200
 - 27s - loss: 0.9056 - acc: 0.7485 - val_loss: 0.8583 - val_acc: 0.7682
Epoch 34/200
 - 26s - loss: 0.8963 - acc: 0.7498 - val_loss: 0.7802 - val_acc: 0.7900
Epoch 35/200
 - 27s - loss: 0.8827 - acc: 0.7535 - val_loss: 0.7781 - val_acc: 0.7902
Epoch 36/200
 - 26s - loss: 0.8764 - acc: 0.7538 - val_loss: 0.7414 - val_acc: 0.8014
Epoch 37/200
 - 27s - loss: 0.8642 - acc: 0.7568 - val_loss: 0.7130 - val_acc: 0.8120
Epoch 38/200
 - 26s - loss: 0.8535 - acc: 0.7593 - val_loss: 0.7010 - val_acc: 0.8102
Epoch 39/200
 - 27s - loss: 0.8422 - acc: 0.7620 - val_loss: 0.7401 - val_acc: 0.7966
Epoch 40/200
 - 26s - loss: 0.8442 - acc: 0.7616 - val_loss: 0.6942 - val_acc: 0.8160
Epoch 41/200
 - 27s - loss: 0.8322 - acc: 0.7667 - val_loss: 0.6851 - val_acc: 0.8172
Epoch 42/200
 - 26s - loss: 0.8182 - acc: 0.7684 - val_loss: 0.6637 - val_acc: 0.8198
Epoch 43/200
 - 27s - loss: 0.8022 - acc: 0.7728 - val_loss: 0.6870 - val_acc: 0.8144
Epoch 44/200
 - 26s - loss: 0.7947 - acc: 0.7746 - val_loss: 0.6695 - val_acc: 0.8210
Epoch 45/200
 - 27s - loss: 0.7947 - acc: 0.7748 - val_loss: 0.6661 - val_acc: 0.8188
Epoch 46/200
 - 26s - loss: 0.7854 - acc: 0.7759 - val_loss: 0.6582 - val_acc: 0.8202
Epoch 47/200
 - 27s - loss: 0.7795 - acc: 0.7779 - val_loss: 0.6648 - val_acc: 0.8198
Epoch 48/200
 - 26s - loss: 0.7769 - acc: 0.7771 - val_loss: 0.6546 - val_acc: 0.8230
Epoch 49/200
 - 27s - loss: 0.7637 - acc: 0.7825 - val_loss: 0.6743 - val_acc: 0.8130
Epoch 50/200
 - 26s - loss: 0.7648 - acc: 0.7806 - val_loss: 0.6502 - val_acc: 0.8208
Epoch 51/200
 - 27s - loss: 0.7578 - acc: 0.7820 - val_loss: 0.6424 - val_acc: 0.8220
Epoch 52/200
 - 26s - loss: 0.7578 - acc: 0.7823 - val_loss: 0.6629 - val_acc: 0.8168
Epoch 53/200
 - 27s - loss: 0.7536 - acc: 0.7837 - val_loss: 0.6120 - val_acc: 0.8304
Epoch 54/200
 - 26s - loss: 0.7483 - acc: 0.7851 - val_loss: 0.6204 - val_acc: 0.8284
Epoch 55/200
 - 27s - loss: 0.7419 - acc: 0.7857 - val_loss: 0.6006 - val_acc: 0.8354
Epoch 56/200
 - 26s - loss: 0.7400 - acc: 0.7854 - val_loss: 0.6247 - val_acc: 0.8238
Epoch 57/200
 - 27s - loss: 0.7313 - acc: 0.7891 - val_loss: 0.6138 - val_acc: 0.8322
Epoch 58/200
 - 26s - loss: 0.7318 - acc: 0.7875 - val_loss: 0.6203 - val_acc: 0.8284
Epoch 59/200
 - 27s - loss: 0.7301 - acc: 0.7880 - val_loss: 0.6224 - val_acc: 0.8262
Epoch 60/200
 - 26s - loss: 0.7264 - acc: 0.7878 - val_loss: 0.6297 - val_acc: 0.8246
Epoch 61/200
 - 27s - loss: 0.7196 - acc: 0.7912 - val_loss: 0.5899 - val_acc: 0.8402
Epoch 62/200
 - 26s - loss: 0.7205 - acc: 0.7916 - val_loss: 0.6238 - val_acc: 0.8238
Epoch 63/200
 - 27s - loss: 0.7175 - acc: 0.7923 - val_loss: 0.5896 - val_acc: 0.8374
Epoch 64/200
 - 26s - loss: 0.7086 - acc: 0.7939 - val_loss: 0.5948 - val_acc: 0.8374
Epoch 65/200
 - 27s - loss: 0.7077 - acc: 0.7933 - val_loss: 0.6014 - val_acc: 0.8338
Epoch 66/200
 - 26s - loss: 0.7088 - acc: 0.7946 - val_loss: 0.5745 - val_acc: 0.8436
Epoch 67/200
 - 27s - loss: 0.7024 - acc: 0.7952 - val_loss: 0.5806 - val_acc: 0.8442
Epoch 68/200
 - 26s - loss: 0.6998 - acc: 0.7950 - val_loss: 0.6054 - val_acc: 0.8328
Epoch 69/200
 - 27s - loss: 0.6985 - acc: 0.7964 - val_loss: 0.5920 - val_acc: 0.8344
Epoch 70/200
 - 26s - loss: 0.6951 - acc: 0.7963 - val_loss: 0.5696 - val_acc: 0.8462
Epoch 71/200
 - 27s - loss: 0.6964 - acc: 0.7959 - val_loss: 0.5747 - val_acc: 0.8392
Epoch 72/200
 - 26s - loss: 0.6893 - acc: 0.7984 - val_loss: 0.5673 - val_acc: 0.8408
Epoch 73/200
 - 27s - loss: 0.6820 - acc: 0.7995 - val_loss: 0.5745 - val_acc: 0.8396
Epoch 74/200
 - 26s - loss: 0.6850 - acc: 0.7982 - val_loss: 0.5894 - val_acc: 0.8340
Epoch 75/200
 - 27s - loss: 0.6847 - acc: 0.7978 - val_loss: 0.6063 - val_acc: 0.8294
Epoch 76/200
 - 26s - loss: 0.6756 - acc: 0.8018 - val_loss: 0.5677 - val_acc: 0.8442
Epoch 77/200
 - 27s - loss: 0.6793 - acc: 0.7998 - val_loss: 0.5847 - val_acc: 0.8394
Epoch 78/200
 - 26s - loss: 0.6769 - acc: 0.8011 - val_loss: 0.5715 - val_acc: 0.8412
Epoch 79/200
 - 28s - loss: 0.6743 - acc: 0.7991 - val_loss: 0.5940 - val_acc: 0.8364
Epoch 80/200
 - 26s - loss: 0.6695 - acc: 0.8018 - val_loss: 0.5759 - val_acc: 0.8420
Epoch 81/200
 - 27s - loss: 0.6709 - acc: 0.8038 - val_loss: 0.5711 - val_acc: 0.8398
Epoch 82/200
 - 26s - loss: 0.6640 - acc: 0.8046 - val_loss: 0.5836 - val_acc: 0.8372
Epoch 83/200
 - 27s - loss: 0.6623 - acc: 0.8067 - val_loss: 0.5406 - val_acc: 0.8508
Epoch 84/200
 - 26s - loss: 0.6631 - acc: 0.8046 - val_loss: 0.5729 - val_acc: 0.8390
Epoch 85/200
 - 27s - loss: 0.6608 - acc: 0.8062 - val_loss: 0.5826 - val_acc: 0.8364
Epoch 86/200
 - 26s - loss: 0.6603 - acc: 0.8044 - val_loss: 0.5633 - val_acc: 0.8416
Epoch 87/200
 - 27s - loss: 0.6581 - acc: 0.8052 - val_loss: 0.5523 - val_acc: 0.8456
Epoch 88/200
 - 26s - loss: 0.6565 - acc: 0.8058 - val_loss: 0.5509 - val_acc: 0.8470
Epoch 89/200
 - 27s - loss: 0.6591 - acc: 0.8060 - val_loss: 0.5591 - val_acc: 0.8428
Epoch 90/200
 - 26s - loss: 0.6513 - acc: 0.8081 - val_loss: 0.5419 - val_acc: 0.8496
Epoch 91/200
 - 27s - loss: 0.6468 - acc: 0.8105 - val_loss: 0.5529 - val_acc: 0.8408
Epoch 92/200
 - 26s - loss: 0.6454 - acc: 0.8096 - val_loss: 0.5576 - val_acc: 0.8450
Epoch 93/200
 - 27s - loss: 0.6508 - acc: 0.8062 - val_loss: 0.5474 - val_acc: 0.8500
Epoch 94/200
 - 26s - loss: 0.6390 - acc: 0.8112 - val_loss: 0.5496 - val_acc: 0.8470
Epoch 95/200
 - 27s - loss: 0.6459 - acc: 0.8096 - val_loss: 0.5608 - val_acc: 0.8402
Epoch 96/200
 - 26s - loss: 0.6467 - acc: 0.8087 - val_loss: 0.5651 - val_acc: 0.8396
Epoch 97/200
 - 27s - loss: 0.6351 - acc: 0.8124 - val_loss: 0.5500 - val_acc: 0.8436
Epoch 98/200
 - 26s - loss: 0.6375 - acc: 0.8116 - val_loss: 0.5496 - val_acc: 0.8468
Epoch 99/200
 - 27s - loss: 0.6354 - acc: 0.8114 - val_loss: 0.5522 - val_acc: 0.8448
Epoch 100/200
 - 26s - loss: 0.6276 - acc: 0.8140 - val_loss: 0.6180 - val_acc: 0.8254
Epoch 101/200
 - 27s - loss: 0.6329 - acc: 0.8135 - val_loss: 0.5689 - val_acc: 0.8440
Epoch 102/200
 - 26s - loss: 0.6313 - acc: 0.8131 - val_loss: 0.5282 - val_acc: 0.8522
Epoch 103/200
 - 27s - loss: 0.6242 - acc: 0.8160 - val_loss: 0.5598 - val_acc: 0.8402
Epoch 104/200
 - 26s - loss: 0.6286 - acc: 0.8145 - val_loss: 0.5654 - val_acc: 0.8414
Epoch 105/200
 - 27s - loss: 0.6217 - acc: 0.8166 - val_loss: 0.5593 - val_acc: 0.8422
Epoch 106/200
 - 26s - loss: 0.6217 - acc: 0.8153 - val_loss: 0.6248 - val_acc: 0.8232
Epoch 107/200
 - 27s - loss: 0.6226 - acc: 0.8150 - val_loss: 0.5885 - val_acc: 0.8320
Epoch 108/200
 - 26s - loss: 0.6188 - acc: 0.8169 - val_loss: 0.5569 - val_acc: 0.8406
Epoch 109/200
 - 27s - loss: 0.6171 - acc: 0.8165 - val_loss: 0.5725 - val_acc: 0.8402
Epoch 110/200
 - 26s - loss: 0.6180 - acc: 0.8170 - val_loss: 0.5375 - val_acc: 0.8498
Epoch 111/200
 - 27s - loss: 0.6197 - acc: 0.8169 - val_loss: 0.5362 - val_acc: 0.8480
Epoch 112/200
 - 26s - loss: 0.6187 - acc: 0.8166 - val_loss: 0.5347 - val_acc: 0.8478
Epoch 113/200
 - 27s - loss: 0.6124 - acc: 0.8192 - val_loss: 0.5433 - val_acc: 0.8504
Epoch 114/200
 - 26s - loss: 0.6121 - acc: 0.8195 - val_loss: 0.6051 - val_acc: 0.8276
Epoch 115/200
 - 27s - loss: 0.6125 - acc: 0.8185 - val_loss: 0.5398 - val_acc: 0.8504
Epoch 116/200
 - 26s - loss: 0.6128 - acc: 0.8181 - val_loss: 0.5469 - val_acc: 0.8490
Epoch 117/200
 - 27s - loss: 0.6078 - acc: 0.8208 - val_loss: 0.5268 - val_acc: 0.8548
Epoch 118/200
 - 26s - loss: 0.6054 - acc: 0.8202 - val_loss: 0.5297 - val_acc: 0.8526
Epoch 119/200
 - 27s - loss: 0.6062 - acc: 0.8205 - val_loss: 0.5542 - val_acc: 0.8434
Epoch 120/200
 - 26s - loss: 0.6027 - acc: 0.8216 - val_loss: 0.5339 - val_acc: 0.8506
Epoch 121/200
 - 27s - loss: 0.6031 - acc: 0.8199 - val_loss: 0.5371 - val_acc: 0.8544
Epoch 122/200
 - 26s - loss: 0.5979 - acc: 0.8227 - val_loss: 0.5331 - val_acc: 0.8528
Epoch 123/200
 - 27s - loss: 0.5980 - acc: 0.8221 - val_loss: 0.5540 - val_acc: 0.8470
Epoch 124/200
 - 26s - loss: 0.5998 - acc: 0.8241 - val_loss: 0.5227 - val_acc: 0.8582
Epoch 125/200
 - 27s - loss: 0.5974 - acc: 0.8239 - val_loss: 0.5328 - val_acc: 0.8536
Epoch 126/200
 - 26s - loss: 0.5906 - acc: 0.8249 - val_loss: 0.5429 - val_acc: 0.8520
Epoch 127/200
 - 27s - loss: 0.5927 - acc: 0.8247 - val_loss: 0.5383 - val_acc: 0.8514
Epoch 128/200
 - 26s - loss: 0.5906 - acc: 0.8250 - val_loss: 0.5397 - val_acc: 0.8506
Epoch 129/200
 - 27s - loss: 0.5920 - acc: 0.8256 - val_loss: 0.5284 - val_acc: 0.8510
Epoch 130/200
 - 26s - loss: 0.5947 - acc: 0.8221 - val_loss: 0.5324 - val_acc: 0.8482
Epoch 131/200
 - 27s - loss: 0.5848 - acc: 0.8272 - val_loss: 0.5456 - val_acc: 0.8508
Epoch 132/200
 - 26s - loss: 0.5885 - acc: 0.8262 - val_loss: 0.5145 - val_acc: 0.8588
Epoch 133/200
 - 27s - loss: 0.5869 - acc: 0.8266 - val_loss: 0.5735 - val_acc: 0.8376
Epoch 134/200
 - 26s - loss: 0.5890 - acc: 0.8263 - val_loss: 0.5474 - val_acc: 0.8458
Epoch 135/200
 - 28s - loss: 0.5779 - acc: 0.8301 - val_loss: 0.5503 - val_acc: 0.8444
Epoch 136/200
 - 26s - loss: 0.5860 - acc: 0.8260 - val_loss: 0.5237 - val_acc: 0.8524
Epoch 137/200
 - 28s - loss: 0.5830 - acc: 0.8283 - val_loss: 0.5121 - val_acc: 0.8596
Epoch 138/200
 - 26s - loss: 0.5758 - acc: 0.8304 - val_loss: 0.5414 - val_acc: 0.8478
Epoch 139/200
 - 28s - loss: 0.5832 - acc: 0.8291 - val_loss: 0.5345 - val_acc: 0.8500
Epoch 140/200
 - 26s - loss: 0.5740 - acc: 0.8320 - val_loss: 0.5465 - val_acc: 0.8516
Epoch 141/200
 - 27s - loss: 0.5786 - acc: 0.8280 - val_loss: 0.5853 - val_acc: 0.8402
Epoch 142/200
 - 26s - loss: 0.5751 - acc: 0.8303 - val_loss: 0.5184 - val_acc: 0.8590
Epoch 143/200
 - 27s - loss: 0.5739 - acc: 0.8299 - val_loss: 0.5247 - val_acc: 0.8568
Epoch 144/200
 - 26s - loss: 0.5804 - acc: 0.8289 - val_loss: 0.5137 - val_acc: 0.8584
Epoch 145/200
 - 27s - loss: 0.5743 - acc: 0.8301 - val_loss: 0.5415 - val_acc: 0.8484
Epoch 146/200
 - 26s - loss: 0.5721 - acc: 0.8331 - val_loss: 0.5148 - val_acc: 0.8586
Epoch 147/200
 - 27s - loss: 0.5738 - acc: 0.8308 - val_loss: 0.5376 - val_acc: 0.8548
Epoch 148/200
 - 26s - loss: 0.5740 - acc: 0.8303 - val_loss: 0.5281 - val_acc: 0.8566
Epoch 149/200
 - 27s - loss: 0.5710 - acc: 0.8327 - val_loss: 0.5281 - val_acc: 0.8588
Epoch 150/200
 - 26s - loss: 0.5684 - acc: 0.8333 - val_loss: 0.5097 - val_acc: 0.8650
Epoch 151/200
 - 27s - loss: 0.5706 - acc: 0.8311 - val_loss: 0.5499 - val_acc: 0.8484
Epoch 152/200
 - 26s - loss: 0.5701 - acc: 0.8309 - val_loss: 0.5359 - val_acc: 0.8528
Epoch 153/200
 - 27s - loss: 0.5661 - acc: 0.8325 - val_loss: 0.5132 - val_acc: 0.8598
Epoch 154/200
 - 26s - loss: 0.5663 - acc: 0.8330 - val_loss: 0.5281 - val_acc: 0.8558
Epoch 155/200
 - 27s - loss: 0.5594 - acc: 0.8359 - val_loss: 0.5050 - val_acc: 0.8626
Epoch 156/200
 - 26s - loss: 0.5612 - acc: 0.8366 - val_loss: 0.5495 - val_acc: 0.8504
Epoch 157/200
 - 28s - loss: 0.5630 - acc: 0.8339 - val_loss: 0.5255 - val_acc: 0.8576
Epoch 158/200
 - 26s - loss: 0.5592 - acc: 0.8345 - val_loss: 0.5219 - val_acc: 0.8588
Epoch 159/200
 - 27s - loss: 0.5596 - acc: 0.8360 - val_loss: 0.5445 - val_acc: 0.8484
Epoch 160/200
 - 26s - loss: 0.5581 - acc: 0.8366 - val_loss: 0.5137 - val_acc: 0.8588
Epoch 161/200
 - 27s - loss: 0.5555 - acc: 0.8337 - val_loss: 0.5272 - val_acc: 0.8522
Epoch 162/200
 - 26s - loss: 0.5621 - acc: 0.8361 - val_loss: 0.5039 - val_acc: 0.8628
Epoch 163/200
 - 27s - loss: 0.5596 - acc: 0.8348 - val_loss: 0.5090 - val_acc: 0.8608
Epoch 164/200
 - 26s - loss: 0.5562 - acc: 0.8375 - val_loss: 0.5400 - val_acc: 0.8482
Epoch 165/200
 - 27s - loss: 0.5505 - acc: 0.8392 - val_loss: 0.5220 - val_acc: 0.8596
Epoch 166/200
 - 27s - loss: 0.5530 - acc: 0.8362 - val_loss: 0.5317 - val_acc: 0.8560
Epoch 167/200
 - 27s - loss: 0.5495 - acc: 0.8384 - val_loss: 0.5231 - val_acc: 0.8552
Epoch 168/200
 - 26s - loss: 0.5549 - acc: 0.8386 - val_loss: 0.5448 - val_acc: 0.8510
Epoch 169/200
 - 27s - loss: 0.5493 - acc: 0.8383 - val_loss: 0.5519 - val_acc: 0.8448
Epoch 170/200
 - 26s - loss: 0.5519 - acc: 0.8387 - val_loss: 0.5261 - val_acc: 0.8526
Epoch 171/200
 - 27s - loss: 0.5472 - acc: 0.8392 - val_loss: 0.5393 - val_acc: 0.8560
Epoch 172/200
 - 26s - loss: 0.5455 - acc: 0.8397 - val_loss: 0.5397 - val_acc: 0.8528
Epoch 173/200
 - 27s - loss: 0.5468 - acc: 0.8410 - val_loss: 0.5229 - val_acc: 0.8602
Epoch 174/200
 - 26s - loss: 0.5432 - acc: 0.8419 - val_loss: 0.5390 - val_acc: 0.8516
Epoch 175/200
 - 27s - loss: 0.5436 - acc: 0.8394 - val_loss: 0.5427 - val_acc: 0.8522
Epoch 176/200
 - 26s - loss: 0.5435 - acc: 0.8410 - val_loss: 0.5197 - val_acc: 0.8538
Epoch 177/200
 - 27s - loss: 0.5443 - acc: 0.8397 - val_loss: 0.5134 - val_acc: 0.8652
Epoch 178/200
 - 26s - loss: 0.5439 - acc: 0.8403 - val_loss: 0.5437 - val_acc: 0.8480
Epoch 179/200
 - 27s - loss: 0.5402 - acc: 0.8426 - val_loss: 0.5170 - val_acc: 0.8590
Epoch 180/200
 - 26s - loss: 0.5372 - acc: 0.8442 - val_loss: 0.5350 - val_acc: 0.8540
Epoch 181/200
 - 27s - loss: 0.5409 - acc: 0.8419 - val_loss: 0.5338 - val_acc: 0.8558
Epoch 182/200
 - 26s - loss: 0.5355 - acc: 0.8442 - val_loss: 0.5213 - val_acc: 0.8618
Epoch 183/200
 - 27s - loss: 0.5374 - acc: 0.8420 - val_loss: 0.5097 - val_acc: 0.8640
Epoch 184/200
 - 26s - loss: 0.5389 - acc: 0.8414 - val_loss: 0.5164 - val_acc: 0.8612
Epoch 185/200
 - 27s - loss: 0.5326 - acc: 0.8446 - val_loss: 0.5163 - val_acc: 0.8642
Epoch 186/200
 - 26s - loss: 0.5327 - acc: 0.8447 - val_loss: 0.5487 - val_acc: 0.8488
Epoch 187/200
 - 27s - loss: 0.5369 - acc: 0.8439 - val_loss: 0.5205 - val_acc: 0.8562
Epoch 188/200
 - 26s - loss: 0.5348 - acc: 0.8423 - val_loss: 0.5116 - val_acc: 0.8630
Epoch 189/200
 - 27s - loss: 0.5266 - acc: 0.8467 - val_loss: 0.5228 - val_acc: 0.8598
Epoch 190/200
 - 26s - loss: 0.5311 - acc: 0.8457 - val_loss: 0.5212 - val_acc: 0.8598
Epoch 191/200
 - 27s - loss: 0.5275 - acc: 0.8465 - val_loss: 0.5242 - val_acc: 0.8578
Epoch 192/200
 - 26s - loss: 0.5284 - acc: 0.8467 - val_loss: 0.5274 - val_acc: 0.8540
Epoch 193/200
 - 27s - loss: 0.5314 - acc: 0.8442 - val_loss: 0.5283 - val_acc: 0.8592
Epoch 194/200
 - 26s - loss: 0.5290 - acc: 0.8453 - val_loss: 0.5072 - val_acc: 0.8634
Epoch 195/200
 - 28s - loss: 0.5301 - acc: 0.8447 - val_loss: 0.5357 - val_acc: 0.8562
Epoch 196/200
 - 26s - loss: 0.5228 - acc: 0.8479 - val_loss: 0.5401 - val_acc: 0.8554
Epoch 197/200
 - 27s - loss: 0.5304 - acc: 0.8448 - val_loss: 0.5040 - val_acc: 0.8648
Epoch 198/200
 - 26s - loss: 0.5218 - acc: 0.8471 - val_loss: 0.5544 - val_acc: 0.8556
Epoch 199/200
 - 27s - loss: 0.5221 - acc: 0.8483 - val_loss: 0.5347 - val_acc: 0.8534
Epoch 200/200
 - 26s - loss: 0.5234 - acc: 0.8477 - val_loss: 0.5294 - val_acc: 0.8566

Test result: 0.842 loss: 0.577